# Measuring Quantum Volume

![hero:quantum volume](images/quantum_volume.png)


## Why quantum volume?

Comparing different quantum computers is not an easy task. There are many different factors that affect the performance of a quantum computer and it’s not always clear which factors will have the biggest effect when doing useful quantum computations.

Say you were designing your quantum computer: Would you trade higher fidelity gates (with less errors) for reduced qubit connectivity (requiring longer circuits)? Maybe you have a great [transpiler](gloss:transpiler) that can organise the swaps so reduced connectivity isn’t as much of an issue, or maybe the readout errors are so high that any improvement in gate fidelity is negligible anyway? We can add each of these statistics to our quantum computer scorecard, but the exact importance of each element is up for debate. Additionally, even if we scored highly at each individual metric, there could still be other unforeseen problems that arise when running quantum circuits.

One thing is certain, we need many qubits and the ability to manipulate them precisely in order to achieve a [quantum advantage](gloss:quantumadvantage). If we do not have enough qubits, we cannot do any useful computations, and if these qubits are not precise or reliable enough, then our measurements will be meaningless.


<div class="tabset">

## What affects computing power?


<!-- ::: tab -->

### Qubit count

**Number of qubits (more is better)**

The number of qubits in our quantum computer sets an upper limit on the power of the computations it can do. To gain advantage over classical computers, we want to be able to entangle as many qubits as possible.

<img src="images/qubit_count.png" width="195"/>

<!-- ::: tab -->

### Readout errors

**Gate and readout errors (less is better)**

As mentioned above, the errors in our quantum computer also set an upper limit on the power of our quantum computer. If we can’t reliably manipulate our quantum states, then we’re not going to get meaningful results from our quantum computer.

    figure: x-img(src="images/readout_errors.png" width=194 height=197)

<!-- ::: tab -->

### Connectivity

**Qubit-Qubit connectivity (more is better)**

If two qubits cannot communicate directly, we need to add extra ‘swap’ gates to move them to the right place, increasing the length of our computation. Quantum volume circuits assume full connectivity, so if you need to insert extra swap gates, that’s on you!

    figure: x-img(src="images/connectivity.png" width=199 height=189)

<!-- ::: tab -->

### Gate set

**Gate set (larger / more powerful is better)**

Quantum volume asks for random two-qubit gates, but working out how to implement them is also on you. If your hardware supports more gates, you’re more likely to get decent results.

    figure: x-img(src="images/gate_set.png" width=302 height=140)

<!-- ::: tab -->

### Software stack

**Compilers and software stack (more intelligent is better)**

Some of the other problems (inserting extra swap gates, creating the specified gates from your available gate set, and dealing with noise) can be mitigated using intelligent classical software such as the transpiler.

    figure: x-img(src="images/software_stack.png" width=308 height=150)

<!-- ::: -->

</div>


This is where quantum volume comes in. Quantum volume is a holistic benchmark, instead of looking at the details, we simply measure how good the quantum computer is at doing the thing we want it to do: Executing quantum circuits. The quantum volume test creates randomized circuits to a specification, and the ‘score’ of the quantum computer increases with the size of the circuit it can reliably execute.

In this chapter, we will learn:

1.	What quantum volume is.
2.	How to create a random square circuit.
3.	How to see if a device can achieve a certain quantum volume.
4.	How to use Qiskit’s tools to make this easier for us.


## What is Quantum Volume?

Quantum volume (QV) is a single-number metric used to measure the power of a quantum computer. It’s used for near-term devices with a modest number of qubits, and measures the largest random circuit of equal depth and width that can be reliably executed.


<!-- ::: .demopanel -->

## Visual demo

<!-- ::: .demobody -->

![vue:what-is-quantum-volume](images/quantum_volume_2.png)

<!-- ::: -->

<!-- ::: -->


## The Quantum Volume Protocol

### Overview

To perform the quantum volume benchmark, we first create a bunch of circuits of size d. We then simulate the circuits and record the most likely outputs for each. We run these same circuits on the device we’re testing and see how regularly the outputs match those we simulated. If the device produces good enough results, we increase d and start over. The quantum volume of the device is 2 to the power of the largest circuit size our device can produce acceptable results for.


<div class="qv-overview">

<!-- ::: column(width=25) -->

## 1.

<!-- ::: column(width=100) -->

![test](images/overview-1.png)

<!-- ::: column.grow -->

Create a set of random circuits of width and depth d.

<!-- ::: -->


<!-- ::: column(width=25) -->

## 2.

<!-- ::: column(width=100) -->

![test](images/overview-2.png)

<!-- ::: column.grow -->

Simulate the circuits and record which outputs are most likely.

<!-- ::: -->


<!-- ::: column(width=25) -->

## 3.

<!-- ::: column(width=100) -->

![test](images/overview-3.png)

<!-- ::: column.grow -->

Execute the circuits on the device we’re testing and record the results.

<!-- ::: -->


<!-- ::: column(width=25) -->

## 4.

<!-- ::: column(width=100) -->

![test](images/overview-4.png)

<!-- ::: column.grow -->

If the device results are statically close enough to the simulation results, increase d and go to step one. Repeat until failure.

<!-- ::: -->


<!-- ::: column(width=25) -->

## 5.

<!-- ::: column(width=100) -->

![test](images/overview-5.png)

<!-- ::: column.grow -->

The quantum volume is 2**dmax, where dmax is the width and depth of the largest circuit our device can successfully execute.

<!-- ::: -->

</div>



## Square circuits

A square circuit is a circuit of equal width and depth. This roughly approximates the kind of circuits we will want to run on a general-purpose quantum computer, and since the depth and width are equal, they can be defined by a single number.

We know we can build our quantum algorithms with quantum circuits using a polynomial number of two-qubit [unitary gates](gloss:unitarygates). The model we choose has layers of random permutations of the qubit labels, followed by randomly specified two-qubits gates. When we have an odd number of qubits, one of the qubits is idle in each layer.

These random circuits more closely approximate the circuits found in near term quantum algorithms than the circuits found in some of the more traditional algorithms.


<!-- ::: .demopanel -->

## Square circuit demo

<!-- ::: .demobody -->

### Layers and Unitaries

Click on any of the layers to see examples of gates inside of each layer. Each layer affects each qubit exactly once, and all the unitary gates in a layer can be executed in parallel (assuming the device has full qubit connectivity).

    div(data-vue-mount)
        layers-circuit
    
<!--
    div(data-vue-mount)
        layers-circuit(layers=3 lines=3 :gates-config="[[{ q1: 0, q2: 1 }, { q1: 1, q2: 2 }],[{ q1: 0, q2: 1 }, { q1: 0, q2: 1 }],[{ q1: 1, q2: 2 }, { q1: 0, q2: 1 }]]")
-->

<!-- ![vue:layers-circuit](images/layers_unitary.png) -->

<!-- ::: -->

<!-- ::: -->


### Creating a square circuit

To start, let’s have a go at creating our own random, square circuit. The first thing we need to do is decide which qubits our circuit is going to act on. When measuring quantum volume, we can choose whichever qubits on our device we think will give us the best results. Here I’m choosing the first five qubits of an imaginary device. This means our circuit’s depth and width will both be five.


In [64]:
qubit_list = [0,1,2,3,4]
size = len(qubit_list)  # size == width == depth

Next, we need a way to create our random 2-qubit gates. Qiskit provides a tool, <code>random_unitary</code> that creates a random nxn unitary operator for us. This function could generate any unitary, two-qubit operation possible.

In [67]:
from qiskit.quantum_info import random_unitary
random_unitary(4)

Operator([[-0.36858382-0.65597438j, -0.04131923+0.43570119j,
           -0.45819161-0.04420006j,  0.17435486+0.00281941j],
          [-0.3338904 +0.13342044j, -0.22408296-0.43645123j,
           -0.13446159-0.10664243j,  0.44301429+0.63584523j],
          [ 0.44709934+0.23712703j,  0.22318688+0.40743813j,
           -0.35780924+0.36281023j,  0.01533944+0.51784238j],
          [-0.21910977+0.02058212j, -0.21074615+0.55454684j,
            0.70634859+0.00819945j,  0.03172802+0.3156405j ]],
         input_dims=(2, 2), output_dims=(2, 2))

We can use this to create a single layer of our random, square circuit:

In [68]:
from qiskit import QuantumCircuit

qv_layer = QuantumCircuit(size)
for pair in range(size//2):  # number of pairs is size/2
    qubit_indices = qubit_list[pair*2:pair*2+2]
    gate = random_unitary(4)
    qv_layer.append(gate, qubit_indices)

qv_layer.draw()

┌──────────┐
q_0: ────────────┤0         ├
     ┌──────────┐│          │
q_1: ┤0         ├┤          ├
     │  unitary ││          │
q_2: ┤1         ├┤  unitary ├
     └──────────┘│          │
q_3: ────────────┤          ├
                 │          │
q_4: ────────────┤1         ├
                 └──────────┘

Now we can create a layer from a list of qubits, all we need to do is randomly change the order of the qubits and repeat. Python has a built-in function to shuffle lists (note that this works ‘in place’, so the data qubit_lists refers to will change).

In [69]:
from random import shuffle
shuffle(qubit_list)
qubit_list

[1, 4, 3, 0, 2]

<div class="demopanel exercise">

## Exercise

<div class="demobody">
    
Create a function, <code>random_square_circuit(n)</code> that takes an integer <code>n</code>, and returns a random <code>QuantumCircuit</code> with width and depth <code>n</code>, that could be used in a quantum volume experiment. You may use the code shown above in your solution.

[Try in IBM Quantum Lab](https://quantum-computing.ibm.com/jupyter)

</div>
</div>

Qiskit also provides a function, <code>qv_circuits</code>, to create these circuits for us. We pass some lists of qubit indices, and an integer which tells qv_circuits how many random, square circuits we’d like to generate for each list of qubits. Here we’re going to give five different combinations of qubits, which will allow us to test up to a quantum volume of 64.

The function qv_circuits returns two lists of quantum circuits, both identical except the first set of circuits all have measurements at the end, whereas the second set do not. The set including measurements is to be executed on hardware, while the other is for use with simulators.

In [70]:
from qiskit.ignis.verification.quantum_volume import qv_circuits
qubit_lists = [[0,1],          # Qubits used to test QV 4
               [0,1,2],        # QV 8
               [0,1,2,3],      # QV 16
               [0,1,2,3,4],    # QV 32
               [0,1,2,3,4,5]]  # QV 64
ntrials = 100  # number of circuits to generate for each qubit list
qv_qc_list, qv_qc_list_nomeas = qv_circuits(qubit_lists, ntrials)

Here is an example of a circuit created by <code>qv_circuits</code>. Note the circuits are organised by trial, then by qubit lists, so here we are seeing the 0th circuit generated for the 2nd entry in <code>qubit_lists</code>.

In [71]:
qv_qc_list_nomeas[0][2].decompose().draw()  # .decompose() unrolls the circuit one level

┌──────────┐            ┌─────────┐┌──────────┐            ┌──────────┐
q_0: ┤1         ├────────────┤1        ├┤0         ├────────────┤0         ├
     │  su4_753 │┌──────────┐│         ││          │┌──────────┐│  su4_421 │
q_1: ┤0         ├┤0         ├┤  su4_45 ├┤  su4_739 ├┤1         ├┤1         ├
     ├─────────┬┘│          ││         ││          ││          │├──────────┤
q_2: ┤0        ├─┤  su4_111 ├┤0        ├┤1         ├┤  su4_436 ├┤1         ├
     │  su4_33 │ │          │└─────────┘└──────────┘│          ││  su4_466 │
q_3: ┤1        ├─┤1         ├───────────────────────┤0         ├┤0         ├
     └─────────┘ └──────────┘                       └──────────┘└──────────┘

## Running the circuit

So how do we know if the quantum circuit is being executed correctly? The quantum volume protocol is run on a perfect simulator as well as the quantum device we’re testing. We can then compare the simulated results (what the quantum computer should do) to the experimental results (what the quantum computer actually does).